In [1]:
# Map google drive where the survITE functions are saved.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load modules**

Force Tensorflow/Keras to 2.12 version (not the default version used on colab now).

Note: the original code in the paper was built under Tensorflow/Keras. We have migrated it to Tensorflow2 previously. Now the colab uses a newer version, but we can still force it into Tensorflow2.

If you encounter message "restart session", please restart the session and ignore the error message below. After restarting the session, you do not need to run this chunk to install tensorflow2. Please directly jump to the next chunk to check whether the Tensorflow/Keras is 2.12 version now.

In [2]:
!pip install --upgrade tensorflow==2.12.0 keras==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.9 MB/s eta 0:00:00
 

In [1]:
import tensorflow as tf, keras
print("tf:", tf.__version__)
print("keras:", keras.__version__)

tf: 2.12.0
keras: 2.12.0


Check if Tensorflow/Keras is 2.12 version.

In [2]:
# Import tensorflow
# Before getting started ensure that TensorFlow 2.x is installed.
import tensorflow as tf

# Check tensorflow version.
print(tf.__version__)

2.18.0


Import packages:

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

Install survival analysis package from scikit:

In [5]:
pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: osqp
    Found existing installation: osqp 1.0.4
    Uninstalling osqp-1.0.4:
      Successfully uninstalled osqp-1.0.4


In [6]:
from sksurv.metrics import concordance_index_ipcw


*   IMPORTANT: Please load python codes updated by us which were migrated from tensorflow1 to tensorflow2. You will encounter errors if running codes in the original paper because the original codes were coded under old version of tensorflow.


Import survITE codes:

In [7]:
# map the folder that contains survITE codes with migration to TF2
import sys
sys.path.insert(0,"/content/drive/My Drive/survITE_TF2")

functions required:

In [8]:
from utils.utils import f_get_minibatch

survITE function:

In [9]:
from models.survite import SurvITE

In [10]:
import time

from sklearn.model_selection import train_test_split

import os

# **Load simulated data**

### Load training data

In [11]:
npz  = np.load('/content/drive/My Drive/survITE-main/data/tr_data.npz')

In [12]:
tr_x = npz['x'] # 10 covs
tr_a = npz['a'] # binary trt
tr_t = npz['t'] # time
tr_y = npz['y'] # event indicator

In [13]:
# trainin data sample size
tr_x.shape

(5000, 10)

In [14]:
tr_x[1:5,1:10]

array([[-1.88150989, -0.92411839, -1.97943022,  0.120276  ,  0.53987698,
         0.36288722, -1.58928698,  0.06541155,  0.09249244],
       [ 0.87595506, -0.96365153,  1.0872579 , -0.8211187 , -0.54412725,
         0.25265539, -0.206866  ,  0.69947509, -0.53299544],
       [ 2.14532015, -0.14266021, -0.10881529,  0.12737963, -1.3780078 ,
         0.28188463, -0.54003036,  0.48722588,  0.28947028],
       [ 0.1178473 , -0.55296617, -0.8717001 ,  0.37965718,  1.1867133 ,
         0.79850919, -0.54472844,  0.06293139, -1.14941159]])

In [15]:
tr_a.shape

(5000,)

In [16]:
tr_a[1:5,]

array([1, 1, 1, 0])

In [17]:
tr_a.mean(0) # 50% trt assignment

0.5004

In [18]:
tr_y.mean(0) # 46% event rate

0.4614

In [19]:
np.unique(tr_t) # unique times

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30.])

### Load test data

In [20]:
npz  = np.load('/content/drive/My Drive/survITE-main/data/te_data.npz')
te_x = npz['x']
te_a = npz['a']
te_t = npz['t']
te_y = npz['y']

In [21]:
te_x.shape

(5000, 10)

In [22]:
np.unique(te_t)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30.])

In [23]:
te_a.mean(0)

0.509

# **Training**

In [24]:
ipm_type      = 'wasserstein'

weight        = False
is_smoothing  = False
is_treat      = True

is_training   = True#True

OUT_ITERATION = 5
N_tr = 5000
N_te = 5000

if not weight:
    weight_type = 'noweight'
else:
    weight_type  = ''


if ipm_type == 'no_ipm':
    beta       = 0.
else:
    beta   = 1e-3 #1e-3


if is_smoothing:
    gamma  = 1e-3
else:
    gamma  = 0.


lr_rate   = 1e-3
mb_size   = 512

keep_prob = 0.7

seed           = 1234

TMAX           = 30
eval_times     = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


# results_mse_hzrd1 = np.zeros([OUT_ITERATION, TMAX+1])
# results_mse_hzrd0 = np.zeros([OUT_ITERATION, TMAX+1])

# results_mse_surv1 = np.zeros([OUT_ITERATION, TMAX+1])
# results_mse_surv0 = np.zeros([OUT_ITERATION, TMAX+1])

# results_hte_hzrd   = np.zeros([OUT_ITERATION, TMAX+1])
# results_hte_surv   = np.zeros([OUT_ITERATION, TMAX+1])

# results1 = np.zeros([OUT_ITERATION, len(eval_times)])
# results2 = np.zeros([OUT_ITERATION, len(eval_times)])
# results3 = np.zeros([OUT_ITERATION, len(eval_times)])

In [25]:
seed         = 1234
modelname    = 'SurvITE'

In [26]:
tr_y_structured = [(tr_y[i], tr_t[i]) for i in range(len(tr_y))]
tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

te_y_structured = [(te_y[i], te_t[i]) for i in range(len(te_y))]
te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])


TMAX           = 30
eval_times     = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


results_mse_hzrd1 = np.zeros([1, TMAX+1])
results_mse_hzrd0 = np.zeros([1, TMAX+1])

results_mse_surv1 = np.zeros([1, TMAX+1])
results_mse_surv0 = np.zeros([1, TMAX+1])

results_hte_hzrd   = np.zeros([1, TMAX+1])
results_hte_surv   = np.zeros([1, TMAX+1])

results1 = np.zeros([1, len(eval_times)])
results2 = np.zeros([1, len(eval_times)])
results3 = np.zeros([1, len(eval_times)])

In [27]:
tf.compat.v1.disable_eager_execution()

In [28]:
num_Event   = 1

z_dim       = 100
x_dim       = np.shape(tr_x)[1]

num_layers1  = 3
h_dim1       = 100

num_layers2  = 2
h_dim2       = 100


input_dims = {
    'x_dim': x_dim,
    'num_Event': num_Event,
    't_max': TMAX+1
}
network_settings = {
    'z_dim': z_dim,

    # Phi()
    'h_dim1': h_dim1,
    'num_layers1': num_layers1,

    # Hypothesis()
    'h_dim2': h_dim2,
    'num_layers2': num_layers2,

    'active_fn': tf.nn.elu,
    'reg_scale': 0.,
    'ipm_term' : ipm_type,
    'is_treat' : is_treat,
    'is_smoothing': is_smoothing
}


tf.compat.v1.reset_default_graph()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

model = SurvITE(sess, 'SurvITE', input_dims, network_settings)
sess.run(tf.compat.v1.global_variables_initializer())
saver       = tf.compat.v1.train.Saver()

savepath = './{}/surviTE/'.format(modelname)

if not os.path.exists(savepath):
    os.makedirs(savepath)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [29]:
if not is_treat:
    tr_a = np.ones_like(tr_t)
    te_a = np.ones_like(te_t)

from sklearn.model_selection import train_test_split

tr_y = tr_y.reshape([-1,1])
tr_t = tr_t.reshape([-1,1])
tr_a = tr_a.reshape([-1,1])

if is_treat:
    tr_w  = np.ones([np.shape(tr_x)[0], TMAX+1, 2])
else:
    tr_w  = np.ones([np.shape(tr_x)[0], TMAX+1, 1])

tr_x_,va_x, tr_y_,va_y, tr_t_,va_t, tr_a_,va_a, tr_w_,va_w = train_test_split(tr_x, tr_y, tr_t, tr_a, tr_w, test_size=0.2, random_state=seed)

In [30]:
iterations = 20000

avg_tr_loss_total = 0.
avg_tr_loss       = 0.
avg_tr_loss_ipm   = 0.

avg_va_loss_total = 0.
avg_va_loss       = 0.
avg_va_loss_ipm   = 0.

check_step        = 100

min_loss          = 1e+8
max_flag          = 20
stop_flag         = 0

if is_training:
    for itr in range(iterations):
        if beta == 0.:
            x_mb, y_mb, t_mb, a_mb = f_get_minibatch(mb_size, tr_x_, tr_y_, tr_t_, tr_a_)
            _, _, tmp_tr_loss          = model.train_baseline(x_mb, y_mb, t_mb, a_mb, lr_train_=lr_rate, k_prob_=keep_prob)
            avg_tr_loss_total         += tmp_tr_loss/check_step
            avg_tr_loss               += tmp_tr_loss/check_step

            x_mb, y_mb, t_mb, a_mb = f_get_minibatch(min(mb_size, np.shape(va_x)[0]), va_x, va_y, va_t, va_a)
            tmp_va_loss                = model.get_loss_basline(x_mb, y_mb, t_mb, a_mb, k_prob_=keep_prob)
            avg_va_loss_total         += tmp_va_loss/check_step
            avg_va_loss               += tmp_va_loss/check_step

        else:
            x_mb, y_mb, t_mb, a_mb, w_mb = f_get_minibatch(mb_size, tr_x_, tr_y_, tr_t_, tr_a_, tr_w_)
            _, _, tmp_tr_loss_total, tmp_tr_loss, tmp_tr_loss_ipm = model.train(x_mb, y_mb, t_mb, a_mb, w_mb,
                                                                                beta_=beta, gamma_=gamma,
                                                                                lr_train_=lr_rate, k_prob_=keep_prob)
            avg_tr_loss_total         += tmp_tr_loss_total/check_step
            avg_tr_loss               += tmp_tr_loss/check_step
            avg_tr_loss_ipm           += tmp_tr_loss_ipm/check_step

            x_mb, y_mb, t_mb, a_mb, w_mb = f_get_minibatch(min(mb_size, np.shape(va_x)[0]), va_x, va_y, va_t, va_a, va_w)
            tmp_va_loss_total, tmp_va_loss, tmp_va_loss_ipm      = model.get_loss(x_mb, y_mb, t_mb, a_mb, w_mb,
                                                                                  beta_=beta, gamma_=gamma,
                                                                                  k_prob_=1.0)
            avg_va_loss_total         += tmp_va_loss_total/check_step
            avg_va_loss               += tmp_va_loss/check_step
            avg_va_loss_ipm           += tmp_va_loss_ipm/check_step

        if (itr + 1)%check_step == 0:
            stop_flag += 1

            print(
                "ITR {:04d}  | TR: loss_T={:.3f} loss_S={:.3f} loss_IPM={:.3f} | loss_T={:.3f} loss_S={:.3f} loss_IPM={:.3f}".format(
                itr+1, avg_tr_loss_total, avg_tr_loss, avg_tr_loss_ipm, avg_va_loss_total, avg_va_loss, avg_va_loss_ipm)
            )

            if min_loss > avg_va_loss_total:
                min_loss  = avg_va_loss_total
                stop_flag = 0

                saver.save(sess, savepath + 'model_{}{}'.format(ipm_type,weight_type))
                print('model saved...')

            else:
                if stop_flag >= max_flag:
                    break


            avg_tr_loss_total = 0.
            avg_tr_loss = 0.
            avg_tr_loss_ipm = 0.

            avg_va_loss_total = 0.
            avg_va_loss = 0.
            avg_va_loss_ipm = 0.

ITR 0100  | TR: loss_T=16.876 loss_S=11.222 loss_IPM=5654.251 | loss_T=13.316 loss_S=12.151 loss_IPM=1165.201
model saved...
ITR 0200  | TR: loss_T=10.589 loss_S=6.586 loss_IPM=4003.342 | loss_T=7.796 loss_S=6.260 loss_IPM=1536.208
model saved...
ITR 0300  | TR: loss_T=9.408 loss_S=6.315 loss_IPM=3093.099 | loss_T=7.320 loss_S=6.119 loss_IPM=1201.219
model saved...
ITR 0400  | TR: loss_T=8.680 loss_S=6.130 loss_IPM=2550.581 | loss_T=6.924 loss_S=6.035 loss_IPM=888.736
model saved...
ITR 0500  | TR: loss_T=8.110 loss_S=5.978 loss_IPM=2131.154 | loss_T=6.550 loss_S=5.875 loss_IPM=675.340
model saved...
ITR 0600  | TR: loss_T=7.660 loss_S=5.865 loss_IPM=1795.137 | loss_T=6.300 loss_S=5.771 loss_IPM=528.613
model saved...
ITR 0700  | TR: loss_T=7.334 loss_S=5.848 loss_IPM=1485.212 | loss_T=6.213 loss_S=5.791 loss_IPM=421.714
model saved...
ITR 0800  | TR: loss_T=7.023 loss_S=5.803 loss_IPM=1219.910 | loss_T=6.057 loss_S=5.721 loss_IPM=335.966
model saved...
ITR 0900  | TR: loss_T=6.699 los

# **Test**

In [31]:
saver.restore(sess, savepath + 'model_{}{}'.format(ipm_type,weight_type))

In [32]:
surv1 = model.predict_survival_A1(te_x)
surv0 = model.predict_survival_A0(te_x)

hzrd1 = model.predict_hazard_A1(te_x)
hzrd0 = model.predict_hazard_A0(te_x)

hzrd = np.zeros_like(hzrd1)
if is_treat:
    hzrd[te_a == 0, :] = hzrd0[te_a == 0, :]
hzrd[te_a == 1, :] = hzrd1[te_a == 1, :]

surv = np.zeros_like(surv1)
if is_treat:
    surv[te_a == 0, :] = surv0[te_a == 0, :]
surv[te_a == 1, :] = surv1[te_a == 1, :]

### Counterfactual survival curves

In [33]:
surv1

array([[0.9999999 , 0.9996264 , 0.99957263, ..., 0.143951  , 0.09930096,
        0.07327607],
       [1.        , 0.9975829 , 0.99741906, ..., 0.9756015 , 0.9756015 ,
        0.97559184],
       [0.9999891 , 0.99508595, 0.99371517, ..., 0.14518113, 0.13489287,
        0.12491893],
       ...,
       [0.999998  , 0.9885373 , 0.98179054, ..., 0.7562634 , 0.7562594 ,
        0.75495607],
       [0.99998903, 0.9973668 , 0.9942506 , ..., 0.1636235 , 0.15260297,
        0.1473879 ],
       [0.99999994, 0.99822724, 0.99801797, ..., 0.7985034 , 0.7956726 ,
        0.77890366]], dtype=float32)

In [34]:
surv0

array([[0.99999994, 0.9994006 , 0.9977398 , ..., 0.08905774, 0.08687469,
        0.08561227],
       [1.        , 0.99883604, 0.9931555 , ..., 0.95973116, 0.9597129 ,
        0.95970947],
       [0.9999874 , 0.99267405, 0.98165905, ..., 0.17758898, 0.16193666,
        0.15525089],
       ...,
       [0.999999  , 0.9836717 , 0.9526803 , ..., 0.7341831 , 0.73378986,
        0.73355025],
       [0.9999825 , 0.9931385 , 0.9790908 , ..., 0.18241341, 0.1446415 ,
        0.1256934 ],
       [1.        , 0.9991685 , 0.9981042 , ..., 0.8046496 , 0.78420687,
        0.7830007 ]], dtype=float32)

In [35]:
hzrd1

array([[1.00047323e-07, 3.73488787e-04, 5.37413543e-05, ...,
        1.50586665e-01, 3.10175270e-01, 2.62080878e-01],
       [1.55583671e-10, 2.41710641e-03, 1.64273632e-04, ...,
        1.03671371e-09, 1.81513649e-08, 9.90773060e-06],
       [1.09037655e-05, 4.90318565e-03, 1.37759349e-03, ...,
        8.24905261e-02, 7.08651021e-02, 7.39396662e-02],
       ...,
       [2.01669604e-06, 1.14606479e-02, 6.82501495e-03, ...,
        9.12303221e-06, 5.28622923e-06, 1.72334223e-03],
       [1.09865377e-05, 2.62224884e-03, 3.12439841e-03, ...,
        4.45975065e-02, 6.73529357e-02, 3.41740549e-02],
       [3.12130162e-08, 1.77269778e-03, 2.09630685e-04, ...,
        8.42430629e-03, 3.54509824e-03, 2.10751239e-02]], dtype=float32)

In [36]:
hzrd0

array([[4.6799482e-08, 5.9931877e-04, 1.6618280e-03, ..., 3.1521493e-01,
        2.4512745e-02, 1.4531360e-02],
       [4.8243375e-11, 1.1639551e-03, 5.6871800e-03, ..., 7.0710470e-08,
        1.9001574e-05, 3.5906792e-06],
       [1.2600348e-05, 7.3134811e-03, 1.1096314e-02, ..., 1.2328279e-01,
        8.8137917e-02, 4.1286282e-02],
       ...,
       [1.0381427e-06, 1.6327281e-02, 3.1505879e-02, ..., 2.7779647e-04,
        5.3554872e-04, 3.2658054e-04],
       [1.7502090e-05, 6.8440791e-03, 1.4144692e-02, ..., 2.2371935e-02,
        2.0706764e-01, 1.3100047e-01],
       [2.0573212e-08, 8.3149254e-04, 1.0651900e-03, ..., 7.7978279e-03,
        2.5405753e-02, 1.5380783e-03]], dtype=float32)

### Factual survival curve

In [37]:
surv

array([[0.99999994, 0.9994006 , 0.9977398 , ..., 0.08905774, 0.08687469,
        0.08561227],
       [1.        , 0.99883604, 0.9931555 , ..., 0.95973116, 0.9597129 ,
        0.95970947],
       [0.9999891 , 0.99508595, 0.99371517, ..., 0.14518113, 0.13489287,
        0.12491893],
       ...,
       [0.999999  , 0.9836717 , 0.9526803 , ..., 0.7341831 , 0.73378986,
        0.73355025],
       [0.99998903, 0.9973668 , 0.9942506 , ..., 0.1636235 , 0.15260297,
        0.1473879 ],
       [1.        , 0.9991685 , 0.9981042 , ..., 0.8046496 , 0.78420687,
        0.7830007 ]], dtype=float32)

In [38]:
hzrd

array([[4.67994816e-08, 5.99318766e-04, 1.66182802e-03, ...,
        3.15214932e-01, 2.45127454e-02, 1.45313600e-02],
       [4.82433746e-11, 1.16395508e-03, 5.68717998e-03, ...,
        7.07104704e-08, 1.90015744e-05, 3.59067917e-06],
       [1.09037655e-05, 4.90318565e-03, 1.37759349e-03, ...,
        8.24905261e-02, 7.08651021e-02, 7.39396662e-02],
       ...,
       [1.03814273e-06, 1.63272806e-02, 3.15058790e-02, ...,
        2.77796469e-04, 5.35548723e-04, 3.26580543e-04],
       [1.09865377e-05, 2.62224884e-03, 3.12439841e-03, ...,
        4.45975065e-02, 6.73529357e-02, 3.41740549e-02],
       [2.05732125e-08, 8.31492536e-04, 1.06518995e-03, ...,
        7.79782794e-03, 2.54057534e-02, 1.53807830e-03]], dtype=float32)